# MOdelo de un renovación urbana
La ciudad de Erstville enfrenta un grave recorte de presupuesto. Buscando una solucion a largo plazo para mejorar la base tributaria, el consejo de la ciudad propone la demolicion de un area de viviendas dentro de la ciudad, y su reemplazo con un moderno desarrollo. El proyecto implica dos fases: (1) demolicion de casas populares para obtener el terreno para el nuevo desarrollo, y (2) construccion del nuevo desarrollo. A continuacion, un resumen de la situacion.

1. Se pueden demoler 300 casas populares. Cada casa ocupa un lote de 0.25 acres. El costo de demoler una casa es de $2000
2. Los tamanios de los lotes para construir casas unifamiliares, dobles, triples y cuadruples, son de 0.18, 0.28, 0.4 y 0.5 acres, respectivamente. Las calles, los espacios abiertos y el area para la instalacion de servicios, ocupan 15% del area disponible.
3. En el nuevo desarrollo, las unidades triples y cuadruples ocupan por lo menos 25% del total. Las unidades sencillas deben ser al menos 20% de todas las unidades, y las unidades dobles deben ocupar un minimo de 10%.
4. El impuesto por unidad aplicado a las unidades sencillas, dobles, triples y cuadruples es de $1000, $1900, $2700 y $3400, respectivamente.
5. El costo de construccion por unidad de las casas sencillas, dobles, triples y cuadruples es de $50,000, $70,000, $130,000 y $160,000, respectivamente. El financiamiento a traves de un banco local esta limitado a $15 millones.

## Pregunta a resolver
¿Cuantas unidades de cada tipo se deben construir para maximizar la recaudacion de impuestos? Debera hacer lo siguiente:

## Requisitos

1. Formular el problema de programamcion lineal.
2. Resolver el problema usando la librerıa JuMP o Pulp, en variables continuas, y determinar la distribucion optima.
3. Resolver el problema en variables enteras, y comparar la solucion con la de (b). Discuta sus resultados.

# Planteamiento del problema

Para hacer este ejercicio es encesario convertir l ainformación textual ecuaciones lineales que se puedan resolver. Los componentes que necesitamos para hacer el ejercicio son las variables, función objetivo y las restricciones. 


### Variables

* x1: Número de unidades de casas sencillas
* x2: Número de unidades de casas dobles
* x3: Número de unidades de casas triples
* x4: Número de unidades de casas cuádruples

### Función objetivo
Dado que lo que se quiere maximizar es la recaudación de impuestos, esto se hace a través de la cantidad óptima de cada tipo de casas múltiplicada por los impuestos de las mismas.

* **F = 1000x1 + 1900x2 + 2700x3 + 3400x4**


### Restricciones
En este caso hay que tener en cuenta bastantes restricciones de hecho. Hay restricciones para el area que se puede utilizar para construir, el presupuesto disponible (teniendo en cuenta los múltiples costos tanto de construccíon como demolición), proporción de unidades y las condiciones triviales.

#### Restricciones triviales
Hay que hacerlo tanto con cantidades continuas como con variables enteras. Pero siempre la cantidad de las casas tiene que ser mayor o igual a 0, osea la restricción de no negatividad.

* **x1,x2,x3,x4 >= 0**

#### Restricción del terreno

Hay espacio de 300 casas y cada una ocupa 0.25 acres, pero de ese total, solo se puede usar el 85% (0.85) ya que 15% será para calles y otras areas no habitables. Entonces:

300 * 0.25 * 0.85 = 63.75 acres es el area disponible

Ojo con algo importante demoler 300 casas tiene un costo. 300 * 2000 = 600000, esto se debe tener en cuenta en el financiamiento.

Tomando los tamaños de lotes por casa sería entonces:

* **0.18x1 + 0.28x2 + 0.40x3 + 0.50x4 <= 63.75**


#### Restricción de financiamiento

Se tiene un financiamiento de 15 millones, pero hay que tener en cuenta costos de demolición y construcción.

costo_dem + costo_constr <= financiamiento

600000 + 50000x1 + 70000x2 + 130000x3 + 160000x4 <= 15000000

* **50000x1 + 70000x2 + 130000x3 + 160000x4 <= 14400000**

#### Restricciones de proporciones

Esta son un poco más detalladas, y aque hay algunas casas que no deben de ocupar tanto espacio del total

Las unidades triples y cuadruples ocupan por lo menos 25% del total, osea:

* __x3 + x4 >= 0.25*(x1 + x2 + x3 + x4)__


Las unidades sencillas deben ser al menos 20% de todas las unidades

* __x1 >= 0.20*(x1 + x2 + x3 + x4)__


las unidades dobles deben ocupar un minimo de 10%

* __x2 >= 0.10*(x1 + x2 + x3 + x4)__



Y con todo esto junto, enntonces ya se puede realizar el modelo utilizando las librerías de Julia.


In [1]:
# importar las librerias que se van a usra
using JuMP
using Ipopt 
using HiGHS 
using Optimization

##### Resolver el problema usando la librerıa JuMP o Pulp, en variables continuas, y determinar la distribucion optima.

In [ ]:
# Define LP model
model_continuo = Model()

# variables de las casas y su restriccion base
@variable(model_continuo, x1 >= 0)
@variable(model_continuo, x2 >= 0)
@variable(model_continuo, x3 >= 0)
@variable(model_continuo, x4 >= 0)

#constraints
# restriccion de terreno
@constraint(model_continuo, 0.18x1 + 0.28x2 + 0.40x3 + 0.50x4 <= 63.75)

# restriccion de financiamiento
@constraint(model_continuo, 50000x1 + 70000x2 + 130000x3 + 160000x4 <= 14400000)

#restricciones de proporciones
@constraint(model_continuo, x3 + x4 >= 0.25*(x1 + x2 + x3 + x4) )
@constraint(model_continuo, x1 >= 0.20*(x1 + x2 + x3 + x4) )
@constraint(model_continuo, x2 >= 0.10*(x1 + x2 + x3 + x4) )

# funcion objetivo
@objective(model_continuo, Max, 1000x1 + 1900x2 + 2700x3 + 3400x4)


1000 x1 + 1900 x2 + 2700 x3 + 3400 x4

In [3]:
set_optimizer(model_continuo, HiGHS.Optimizer)

In [4]:
model_continuo

A JuMP Model
├ solver: HiGHS
├ objective_sense: MAX_SENSE
│ └ objective_function_type: AffExpr
├ num_variables: 4
├ num_constraints: 9
│ ├ AffExpr in MOI.GreaterThan{Float64}: 3
│ ├ AffExpr in MOI.LessThan{Float64}: 2
│ └ VariableRef in MOI.GreaterThan{Float64}: 4
└ Names registered in the model
  └ :x1, :x2, :x3, :x4

In [5]:
optimize!(model_continuo)

Running HiGHS 1.11.0 (git hash: 364c83a51e): Copyright (c) 2025 HiGHS under MIT licence terms
LP   has 5 rows; 4 cols; 20 nonzeros
Coefficient ranges:
  Matrix [1e-01, 2e+05]
  Cost   [1e+03, 3e+03]
  Bound  [0e+00, 0e+00]
  RHS    [6e+01, 1e+07]
Presolving model
5 rows, 4 cols, 20 nonzeros  0s
5 rows, 4 cols, 20 nonzeros  0s
Presolve : Reductions: rows 5(-0); columns 4(-0); elements 20(-0) - Not reduced
Problem not reduced by presolve: solving the LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -1.2578114611e+02 Ph1: 4(14.302); Du: 4(125.781) 0s
          3     3.4133333333e+05 Pr: 0(0) 0s
Model status        : Optimal
Simplex   iterations: 3
Objective value     :  3.4133333333e+05
P-D objective error :  0.0000000000e+00
HiGHS run time      :          0.00


In [6]:
# solucion
print("x1 = ", value.(x1), "\n")
print("x2 = ", value.(x2), "\n")
print("x3 = ", value.(x3), "\n")
print("x4 = ", value.(x4), "\n\n")

print("Los impuestos maximizados serian = ", objective_value(model_continuo))

x1 = 35.55555555555556
x2 = 97.77777777777777
x3 = 44.44444444444444
x4 = 0.0

Los impuestos maximizados serian = 341333.3333333333

##### Resolver el problema en variables enteras, y comparar la solucion con la de (b). Discuta sus resultados.

In [ ]:
# Define LP model
model_discreto = Model()

# variables de las casas y su restriccion base
@variable(model_discreto, x1 >= 0, Int)
@variable(model_discreto, x2 >= 0, Int)
@variable(model_discreto, x3 >= 0, Int)
@variable(model_discreto, x4 >= 0, Int)


#constraints
# restriccion de terreno
@constraint(model_discreto, 0.18x1 + 0.28x2 + 0.40x3 + 0.50x4 <= 63.75)

# restriccion de financiamiento
@constraint(model_discreto, 50000x1 + 70000x2 + 130000x3 + 160000x4 <= 14400000)

#restricciones de proporciones
@constraint(model_discreto, x3 + x4 >= 0.25*(x1 + x2 + x3 + x4) )
@constraint(model_discreto, x1 >= 0.20*(x1 + x2 + x3 + x4) )
@constraint(model_discreto, x2 >= 0.10*(x1 + x2 + x3 + x4) )

# funcion objetivo
@objective(model_discreto, Max, 1000x1 + 1900x2 + 2700x3 + 3400x4)

1000 x1 + 1900 x2 + 2700 x3 + 3400 x4

In [8]:
set_optimizer(model_discreto, HiGHS.Optimizer)

In [9]:
model_discreto

A JuMP Model
├ solver: HiGHS
├ objective_sense: MAX_SENSE
│ └ objective_function_type: AffExpr
├ num_variables: 4
├ num_constraints: 13
│ ├ AffExpr in MOI.GreaterThan{Float64}: 3
│ ├ AffExpr in MOI.LessThan{Float64}: 2
│ ├ VariableRef in MOI.GreaterThan{Float64}: 4
│ └ VariableRef in MOI.Integer: 4
└ Names registered in the model
  └ :x1, :x2, :x3, :x4

In [10]:

optimize!(model_discreto)

Running HiGHS 1.11.0 (git hash: 364c83a51e): Copyright (c) 2025 HiGHS under MIT licence terms
MIP  has 5 rows; 4 cols; 20 nonzeros; 4 integer variables (0 binary)
Coefficient ranges:
  Matrix [1e-01, 2e+05]
  Cost   [1e+03, 3e+03]
  Bound  [0e+00, 0e+00]
  RHS    [6e+01, 1e+07]
Presolving model
5 rows, 4 cols, 20 nonzeros  0s
5 rows, 4 cols, 20 nonzeros  0s
Objective function is integral with scale 0.01

Solving MIP model with:
   5 rows
   4 cols (0 binary, 4 integer, 0 implied int., 0 continuous, 0 domain fixed)
   20 nonzeros

Src: B => Branching; C => Central rounding; F => Feasibility pump; J => Feasibility jump;
     H => Heuristic; L => Sub-MIP; P => Empty MIP; R => Randomized rounding; Z => ZI Round;
     I => Shifting; S => Solve LP; T => Evaluate node; U => Unbounded; X => User solution;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |   

In [11]:
# solucion
print("x1 = ", value.(x1), "\n")
print("x2 = ", value.(x2), "\n")
print("x3 = ", value.(x3), "\n")
print("x4 = ", value.(x4), "\n\n")

print("Los impuestos maximizados serian = ", objective_value(model_discreto))


x1 = 34.99999999999991
x2 = 96.00000000000001
x3 = 37.000000000002224
x4 = 6.999999999998213

Los impuestos maximizados serian = 341099.9999999999

A pesar de haber especificado que fueran cantidades enteras aun hay decimales, sin embargo esto parece ser más un error de cálculo de punto flotante, ya que los decimales extra son muy pequños. Entonces se podría aproximar de la siguiente forma:

* x1 = 35
* x2 = 96
* x3 = 37
* x4 = 7

Evaluando en la función objetivo se obtiene:

1000*(35) + 1900*(96) + 2700*(37) + 3400*(7) = **341100**


# Comparación de resultados

## Cantidad de casas a construir
| Variable | Respuesta Continua | Respuesta Discreta |
| -------- | :------------------: | :-----------------: |
| January  | 35.55555555555556  |        35         |
| February | 97.77777777777777  |        96         |
| March    | 44.44444444444444  |        37         |
| March    | 0.0                |         7         |


## Impuestos maximizados
|          | Respuesta Continua | Respuesta Discreta |
| -------- |:------------------:| :-----------------:|
|  Total   | 341333.3333333333  |       341100       |


Como se puede ver al final realmente los resultados son un tanto parecidos, siendo $341,333 aproxmadamente cuando se permite que la cantidad de casas sea continua y $341,100, habiendo entonces una diferencia de $233. Realmente la diferencia no es mucha y de tomar una decisión sería mejor tomar en cuenta el modelo con respuetas discretas. Lo que pasa es que realmente no es práctico decir que hay que hacer 44.4 casas triples. En un escenario real, las casas no se pueden hacer a medias. Es mejor tomar en cuenta el modelo que provee la cantidad en enteros de las casas. Halgo que me llama la atención es que la cantidad discreta parece más real, ya que en este si se construyn casas de todos los tipos. Se hacen menos casas triples pero si se construyen casas cuadruples, la diferencia de los impuestos al final no es tan significativa entonces es mejor tomar la cantidad discreta de casas, además que en este hay más variedad. 
